In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
import os

In [4]:
path = tf.keras.utils.get_file(
'nietzsche.txt',
origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

606208/600901 [==============================] - 0s 1us/step


In [5]:
text = open(path).read()
print(len(text))
print(text[:500])

600893
PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to Truth, have been unskilled and unseemly methods for
winning a woman? Certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--IF,
indeed, it s


In [6]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [7]:

sentences[0]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is the'

In [8]:
next_chars[0]

'r'

In [9]:
len(sentences)

200278

In [10]:
chars = sorted(list(set(text)))
vocab_len = len(chars)

In [11]:
char_indices = dict((char, chars.index(char)) for char in chars)
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 'A': 24,
 'B': 25,
 'C': 26,
 'D': 27,
 'E': 28,
 'F': 29,
 'G': 30,
 'H': 31,
 'I': 32,
 'J': 33,
 'K': 34,
 'L': 35,
 'M': 36,
 'N': 37,
 'O': 38,
 'P': 39,
 'Q': 40,
 'R': 41,
 'S': 42,
 'T': 43,
 'U': 44,
 'V': 45,
 'W': 46,
 'X': 47,
 'Y': 48,
 'Z': 49,
 '[': 50,
 ']': 51,
 '_': 52,
 'a': 53,
 'b': 54,
 'c': 55,
 'd': 56,
 'e': 57,
 'f': 58,
 'g': 59,
 'h': 60,
 'i': 61,
 'j': 62,
 'k': 63,
 'l': 64,
 'm': 65,
 'n': 66,
 'o': 67,
 'p': 68,
 'q': 69,
 'r': 70,
 's': 71,
 't': 72,
 'u': 73,
 'v': 74,
 'w': 75,
 'x': 76,
 'y': 77,
 'z': 78,
 'Æ': 79,
 'ä': 80,
 'æ': 81,
 'é': 82,
 'ë': 83}

In [12]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [13]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [14]:
y[100]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [15]:
x[100].shape

(60, 84)

In [16]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, vocab_len)))

In [17]:
model.add(layers.Dense(vocab_len, activation="softmax"))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               109056    
_________________________________________________________________
dense (Dense)                (None, 84)                10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [22]:
import random
import sys
for epoch in range(0, 60):
    print("Epoch ", epoch)
    f = open("epoch_{}.txt".format(epoch), "w")
    f.write("Epoch {}\n".format(epoch))
    model.fit(x,y, batch_size=128)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    f.write("Seed {}\n".format(generated_text))
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        f.write("Temperature {}\n".format(temperature))
        #print(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
           # sys.stdout.write(next_char)
            f.write(next_char)
    f.close()

Epoch  0
1565/1565 [==============================] - 10s 7ms/step - loss: 2.2131
--- Generating with seed: "To REVERSE all estimates of value--THAT is what they
had to "
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  1
1565/1565 [==============================] - 10s 6ms/step - loss: 2.0674
--- Generating with seed: "ld-renunciation, and will-renunciation, both
symptoms perhap"
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  2
1565/1565 [==============================] - 10s 7ms/step - loss: 1.9681
--- Generating with seed: "They set themselves
before the eyes of all not alone as mode"
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  3
1565/1565 [==============================] - 10s 7ms/step - loss: 1.8946
--- Generating with seed: " whoever, with an Asiatic and super-Asiatic
eye, has actuall"
------ temperature: 0.2
----

------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  31
1565/1565 [==============================] - 10s 7ms/step - loss: 1.3483
--- Generating with seed: "s to "conception" or "notion."


30

=Evil Habits in Reachin"
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  32
1565/1565 [==============================] - 11s 7ms/step - loss: 1.3418
--- Generating with seed: " which is the danger of dangers for highly developed
and wea"
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  33
1565/1565 [==============================] - 11s 7ms/step - loss: 1.3341
--- Generating with seed: ", our laughter itself may have a
future!

224. The historica"
------ temperature: 0.2
------ temperature: 0.5
------ temperature: 1.0
------ temperature: 1.2
Epoch  34
1565/1565 [==============================] - 11s 7ms/step - loss: 1.3272
--- Generating with seed: "teful." Let us

In [29]:
epoch = 1
_f = open("epoch_{}.txt".format(epoch), "r")

In [40]:
for i in range(0, 10):
    _f = open("epoch_{}.txt".format(i), "w")
    _f.write("epoch {}\n".format(i))
    sr = "rohan"
    for i in sr:
        _f.write(i)
    _f.close()

In [2]:
ls

__notebook_source__.ipynb


In [1]:
import tensorflow as tf
import os
import numpy as np
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [2]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [3]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [4]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [5]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [6]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [7]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [8]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [9]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [12]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173251


In [16]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [17]:
model.compile(optimizer='adam', loss=loss)
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 8s 44ms/step - loss: 2.6692
Epoch 2/10
172/172 [==============================] - 7s 44ms/step - loss: 1.9716
Epoch 3/10
172/172 [==============================] - 7s 43ms/step - loss: 1.7030
Epoch 4/10
172/172 [==============================] - 8s 44ms/step - loss: 1.5521
Epoch 5/10
172/172 [==============================] - 7s 43ms/step - loss: 1.4642
Epoch 6/10
172/172 [==============================] - 7s 43ms/step - loss: 1.4030
Epoch 7/10
172/172 [==============================] - 7s 43ms/step - loss: 1.3575
Epoch 8/10
172/172 [==============================] - 7s 43ms/step - loss: 1.3193
Epoch 9/10
172/172 [==============================] - 7s 43ms/step - loss: 1.2852
Epoch 10/10
172/172 [==============================] - 8s 44ms/step - loss: 1.2537


In [18]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [19]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 2000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: we!
The wedding-shit Would incense. But for your hagrant;
For him, my good mortak from the ground, Thomouhe by the winds for God us;
To weep it the duke,
Do nd git so.

EDWARD:
Soul he out, I can well knock thy bear?

AUFIDIUS:
How fartoch--

COMINIUS:
Do not this death:
Excellent glack for counte heartily grace:
Thou desire you as his foot fewname wors:
Once more Edward:
The world shall be full of noble heart.

PROSPERO:
Where is thy foels,
I think thyself?

AGHOMAS:
Have to yourself cle-kit your father's carub
of presence that I were, gentle maided:
Too duty, god in this foul fire,
That could perform with wars light to him:
While it is postery. Wed in thy face, I can tell you.

COMINIUS:
The host shall make to blaw.

PAULINA:
How fails upon your addies,
To do to let him well, in the adventious steed
Was to sing and to stay point of torgure in the queen.

BALNANIO:
Fear too woman.

COMINIUS:
Hide me on my faith, nor the crown
In all is ELIZABETH:
The mutinius, Cagens: I was win